In [5]:
import mido

In [20]:
class Note:
    NOTES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    OCTAVES = list(range(11))
    NOTES_IN_OCTAVE = len(NOTES)

    def __init__(self, note, velocity=127, osc=None):
        self.note = note
        self.velocity = velocity
        self.osc = osc

    def __eq__(self, __value: object) -> bool:
        if __value is None:
            return False
        if isinstance(__value, int):
            return self.note == __value
        return self.note == __value.note

    def __lt__(self, __value: object) -> bool:
        return self.note < __value.note

    def __gt__(self, __value: object) -> bool:
        return self.note > __value.note

    def __le__(self, __value: object) -> bool:
        return self.note <= __value.note

    def __ge__(self, __value: object) -> bool:
        return self.note >= __value.note

    def __str__(self):
        note, octave = self.number_to_note(self.note)
        return f'{note}{octave}'

    def __repr__(self) -> str:
        return self.__str__()

    def number_to_note(self, number: int) -> tuple:
        octave = number // self.NOTES_IN_OCTAVE
        note = self.NOTES[number % self.NOTES_IN_OCTAVE]
        return note, octave

In [16]:
class Oscillator:
    octave_dict = {-1: 0, 0: 42, 1: 84, 2: 127}
    def __init__(self, midi_controller, tuning_channel, octave_channel, volume_channel):
        self.midi_controller = midi_controller

        self.tuning_channel = tuning_channel
        self.octave_channel = octave_channel
        self.volume_channel = volume_channel

        self.tuning_semitones = 0
        self.volume = 127

    def set_tuning(self, semitones):
        self.tuning_semitones = semitones

        semitone_diff = semitones
        octave_diff = 0

        if abs(semitone_diff) > 12:
            octave_diff = semitone_diff // 12
            semitone_diff = semitone_diff % 12

        if semitone_diff == 0:
            cv = 64
        elif semitone_diff > 0:
            if semitone_diff == 1:
                cv = 83
            else:
                cv = 83 + 4 * (semitone_diff - 1)
        else:
            if semitone_diff == -1:
                cv = 44
            else:
                cv = 44 + 4 * (semitone_diff + 1)

        self.midi_controller.control_change(self.tuning_channel, cv)
        self.midi_controller.control_change(self.octave_channel, self.octave_dict.get(octave_diff, 42))

    def set_volume(self, volume):
        self.volume = volume
        self.midi_controller.control_change(self.volume_channel, volume)


In [23]:
class MidiController:
    def __init__(self, out_port, in_port):
        self.notes_pressed = []
        self.oscillators = [Oscillator(self, 34, 48, 39), Oscillator(self, 35, 49, 40)]
        self.notes_playing = [None, None] # [osc1, osc2]

        self.out_port = out_port
        self.in_port = in_port

    def control_change(self, control, value):
        self.out_port.send(mido.Message('control_change', control=control, value=value))

    def parse_midi(self, msg):
        if msg.type == 'clock':
            return
        if msg.type == 'note_on' or msg.type == 'note_off':
            note = Note(msg.note, msg.velocity)
            if msg.type == 'note_on':
                self.add_note_pressed(note)
            elif msg.type == 'note_off':
                self.remove_note_pressed(note)

    def add_note_pressed(self, note: Note):
        if note not in self.notes_pressed:
            self.notes_pressed.append(note)
            if note == min(self.notes_pressed) and len(self.notes_pressed) >= 1:
                self.start_note_on_osc1(note)
            else:
                self.start_note_on_osc2(note)

    def remove_note_pressed(self, note: Note):
        if note in self.notes_pressed:
            self.notes_pressed.remove(note)
            if len(self.notes_pressed) == 0:
                self.all_note_off()
            elif note == self.notes_playing[0]:
                self.stop_note_on_osc1()
                if len(self.notes_pressed) > 1:
                    self.start_note_on_osc1(min(self.notes_pressed))
            elif note == self.notes_playing[1]:
                self.stop_note_on_osc2()
                if len(self.notes_pressed) > 1:
                    self.start_note_on_osc2(self.notes_pressed[-1])

    def start_note_on_osc1(self, note: Note):
        print('start_note_on_osc1', note, self.notes_playing, self.notes_pressed)
        if self.notes_playing[1] is None:
            self.oscillators[0].set_tuning(0)
            self.oscillators[0].set_volume(note.velocity)
            self.note_on(note)
        else:
            osc2_note = self.notes_playing[1]
            self.stop_note_on_osc2()
            self.start_note_on_osc1(note) # FIXME this is not the best way to do it, we can detune osc1
            self.start_note_on_osc2(osc2_note)
        self.notes_playing[0] = note

    def stop_note_on_osc1(self):
        print('stop_note_on_osc1')
        self.oscillators[0].set_volume(0)
        if self.notes_playing[1] is None:
            self.all_note_off()
        self.notes_playing[0] = None

    def start_note_on_osc2(self, note: Note):
        print('start_note_on_osc2', note, self.notes_playing, self.notes_pressed)
        if self.notes_playing[0] is None:
            self.oscillators[1].set_tuning(0)
            self.oscillators[1].set_volume(note.velocity)
            self.oscillators[0].set_volume(0)
            self.note_on(note)
        else:
            self.oscillators[1].set_tuning(note.note - self.notes_playing[0].note)
            self.oscillators[1].set_volume(note.velocity)
        self.notes_playing[1] = note

    def stop_note_on_osc2(self):
        print('stop_note_on_osc2')
        self.oscillators[1].set_volume(0)
        if self.notes_playing[0] is None:
            self.all_note_off()
        self.notes_playing[1] = None

    def note_on(self, note: Note):
        self.out_port.send(mido.Message('note_on', note=note.note))

    def all_note_off(self):
        print('all_note_off')
        self.control_change(123, 0)
        self.oscillators[0].set_volume(0)
        self.oscillators[1].set_volume(0)
        self.notes_playing = [None, None]

    def run(self):
        for msg in self.in_port:
            self.parse_midi(msg)

In [9]:
out_port = mido.open_output('monologue SOUND')
in_port = mido.open_input('monologue KBD/KNOB')


In [24]:
controller = MidiController(out_port, in_port)
controller.run()

start_note_on_osc1 A4 [None, None] [A4]
all_note_off
start_note_on_osc1 A4 [None, None] [A4]
all_note_off
start_note_on_osc1 A4 [None, None] [A4]
all_note_off
start_note_on_osc1 A4 [None, None] [A4]
all_note_off
start_note_on_osc1 A4 [None, None] [A4]
all_note_off
start_note_on_osc1 A4 [None, None] [A4]
start_note_on_osc2 B4 [A4, None] [A4, B4]
start_note_on_osc2 C5 [A4, B4] [A4, B4, C5]
start_note_on_osc2 D5 [A4, C5] [A4, C5, D5]
start_note_on_osc2 E5 [A4, D5] [A4, D5, E5]
stop_note_on_osc2
start_note_on_osc2 F5 [A4, None] [A4, F5]
start_note_on_osc2 E5 [A4, F5] [A4, F5, E5]
start_note_on_osc2 D5 [A4, E5] [A4, E5, D5]
start_note_on_osc2 C5 [A4, D5] [A4, D5, C5]
start_note_on_osc2 B4 [A4, C5] [A4, C5, B4]
stop_note_on_osc2
start_note_on_osc2 C5 [A4, None] [A4, C5]
start_note_on_osc2 D5 [A4, C5] [A4, C5, D5]
start_note_on_osc2 C5 [A4, D5] [A4, D5, C5]
stop_note_on_osc1
start_note_on_osc1 B4 [None, C5] [C5, B4]
stop_note_on_osc2
all_note_off
start_note_on_osc1 B4 [None, None] [C5, B4]
st

KeyboardInterrupt: 